In [2]:
import os
import json
import logging

import boto3
import requests
from ingest_flights.utils.models import StateVector
from datetime import datetime

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# AWS clients
secrets_client = boto3.client("secretsmanager")
kinesis_client = boto3.client("kinesis")

# OpenSky OAuth2 + API endpoints
OPENSKY_TOKEN_URL = (
    "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token"
)
OPENSKY_STATES_URL = "https://opensky-network.org/api/states/all"



In [3]:

def _get_opensky_credentials_from_secret():
    """
    Lê client_id e client_secret do AWS Secrets Manager.

    Espera que o secret (apontado por OPENSKY_SECRET_ARN) tenha o formato:
    {
      "client_id": "xxxx",
      "client_secret": "yyyy"
    }
    """
    try:
        client_id = os.environ.get("OPENSKY_client_id")
        client_secret = os.environ.get("OPENSKY_client_secret")
        if not client_id or not client_secret:
            logger.error("Credentials is missing in secret")
            return None, None
        return client_id, client_secret
    except Exception as e:
        logger.error(f"Error retrieving OpenSky credentials from Secrets Manager: {e}")
        return None, None

def get_opensky_access_token():
    """
    Autentica na OpenSky via OAuth2 Client Credentials e retorna o access_token (Bearer).
    """
    client_id, client_secret = _get_opensky_credentials_from_secret()
    if not client_id or not client_secret:
        return None

    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
    }

    try:
        resp = requests.post(OPENSKY_TOKEN_URL, data=data, timeout=10)
        resp.raise_for_status()
        access_token = resp.json().get("access_token")
        if not access_token:
            logger.error("No access_token in OpenSky auth response")
            return None
        logger.info("Successfully obtained OpenSky access token")
        return access_token
    except requests.RequestException as e:
        logger.error(f"Error obtaining OpenSky access token: {e}")
        return None

def convert_states_response_to_json(states: list[StateVector]) -> dict:
    json_data = {
        "timestamp": datetime.now().isoformat(),
        "total_states": 0,
        "states": [],
    }
    for state in states:
        if state.origin_country != "Brazil":
            continue
        json_data["states"].append(state.to_dict())

    json_data["total_states"] = len(json_data["states"])
    return json_data

def send_state_to_kinesis(state_dict: dict) -> bool:
    """Envia um único registro para o Kinesis.

    Lê o nome do stream da variável de ambiente KINESIS_STREAM.
    Retorna True em caso de sucesso, False em caso de erro.
    """
    stream_name = os.environ.get("KINESIS_STREAM")
    if not stream_name:
        logger.error("KINESIS_STREAM environment variable not set")
        return False

    try:
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(state_dict),
            PartitionKey=state_dict.get("icao24") or "unknown",
        )
        logger.info(
            f"State {state_dict.get('icao24', 'unknown')} sent to Kinesis successfully"
        )
        return True
    except Exception as e:
        logger.error(f"Error sending state to Kinesis: {e}")
        return False


def send_states_to_kinesis(json_resultado: dict) -> bool:
    """Envia todos os estados para o Kinesis.

    Retorna True se todos forem enviados com sucesso, False caso ocorra algum erro.
    """
    all_ok = True
    for state in json_resultado["states"]:
        ok = send_state_to_kinesis(state)
        if not ok:
            all_ok = False
    logger.info(
        f"Sent {len(json_resultado['states'])} states to Kinesis stream (check logs for failures)"
    )
    return all_ok


def get_opensky_states(access_token):
    """
    Chama o endpoint /api/states/all usando Bearer token e
    retorna uma lista de StateVector.
    """
    headers = {"Authorization": f"Bearer {access_token}"}

    try:
        resp = requests.get(OPENSKY_STATES_URL, headers=headers, timeout=15)
        resp.raise_for_status()
        body = resp.json()
    except requests.RequestException as e:
        logger.error(f"Error calling OpenSky states API: {e}")
        return []

    raw_states = body.get("states", []) or []
    states: list[StateVector] = []

    for row in raw_states:
        try:
            state = StateVector.from_api_response(row)
            states.append(state)
        except Exception as e:
            logger.warning(f"Failed to parse state vector: {e}")

    logger.info(f"Retrieved {len(states)} state vectors from OpenSky API")
    return states



In [4]:
access_token = get_opensky_access_token()
access_token 

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ0SVIwSDB0bmNEZTlKYmp4dFctWEtqZ0RYSWExNnR5eU5DWHJxUzJQNkRjIn0.eyJleHAiOjE3NzA0ODI1MjQsImlhdCI6MTc3MDQ4MDcyNCwianRpIjoiMjBkZTk0NzQtYmNlYS00ZDAzLWE0ZDMtMmRjMjk2YmI4ZDQ2IiwiaXNzIjoiaHR0cHM6Ly9hdXRoLm9wZW5za3ktbmV0d29yay5vcmcvYXV0aC9yZWFsbXMvb3BlbnNreS1uZXR3b3JrIiwiYXVkIjpbIndlYnNpdGUtdWkiLCJhY2NvdW50Il0sInN1YiI6IjRmZDI3NTg0LTAwZTgtNDRkNC05Y2VhLTljNGFmYzhkZTYzNyIsInR5cCI6IkJlYXJlciIsImF6cCI6ImphbWlsdmlsZWxhLWFwaS1jbGllbnQiLCJhY3IiOiIxIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwiT1BFTlNLWV9BUElfREVGQVVMVCIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1vcGVuc2t5LW5ldHdvcmsiXX0sInJlc291cmNlX2FjY2VzcyI6eyJ3ZWJzaXRlLXVpIjp7InJvbGVzIjpbIm9wZW5za3lfd2Vic2l0ZV91c2VyIl19LCJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6InByb2ZpbGUgZW1haWwiLCJjbGllbnRIb3N0IjoiMTc3LjEyLjEwLjE3IiwiY2xpZW50SWQiOiJqYW1pbHZpbGVsYS1hcGktY2xpZW50IiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJwcmVmZXJyZWR

In [5]:
states = get_opensky_states(access_token)

In [6]:
json_resultado = convert_states_response_to_json(states)

In [7]:
json_resultado

{'timestamp': '2026-02-07T13:12:22.608912',
 'total_states': 127,
 'states': [{'icao24': 'e4943e',
   'callsign': 'GLO7481',
   'origin_country': 'Brazil',
   'time_position': '2026-02-07T13:12:13',
   'last_contact': '2026-02-07T13:12:13',
   'longitude': -54.9593,
   'latitude': -25.547,
   'altitude': 10668.0,
   'on_ground': False,
   'velocity': 238.99,
   'heading': 94.69,
   'vertical_rate': 0.98,
   'sensors': None,
   'geo_altitude': 11353.8,
   'squawk': None,
   'spi': False,
   'position_source': 0},
  {'icao24': 'e4943f',
   'callsign': 'GLO9672',
   'origin_country': 'Brazil',
   'time_position': '2026-02-07T13:12:12',
   'last_contact': '2026-02-07T13:12:12',
   'longitude': -46.7251,
   'latitude': -23.5274,
   'altitude': 1348.74,
   'on_ground': False,
   'velocity': 89.99,
   'heading': 149.04,
   'vertical_rate': -1.95,
   'sensors': None,
   'geo_altitude': 1409.7,
   'squawk': None,
   'spi': False,
   'position_source': 0},
  {'icao24': 'e49443',
   'callsign': '

In [8]:
ok = send_states_to_kinesis(json_resultado)

KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment variable not set
KINESIS_STREAM environment varia